# Classification Surrogate Tests

We are interested in testing whether or not a surrogate model can correctly identify unknown constraints based on categorical criteria with classification surrogates. Essentially, we want to account for scenarios where specialists can look at a set of experiments and label outcomes as 'acceptable', 'unacceptable', 'ideal', etc. 

This involves new models that produce `CategoricalOutput`'s rather than continuous outputs. Mathematically, if $g_{\theta}:\mathbb{R}^d\to[0,1]^c$ represents the function governed by learnable parameters $\theta$ which outputs a probability vector over $c$ potential classes (i.e. for input $x\in\mathbb{R}^d$, $g_{\theta}(x)^\top\mathbf{1}=1$ where $\mathbf{1}$ is the vector of all 1's) and we have acceptibility criteria for the corresponding classes given by $a\in\{0,1\}^c$, we can compute the scalar output $g_{\theta}(x)^\top a\in[0,1]$ which represents the expected value of acceptance as an objective value to be passed in as a constrained function.

In this script, we look at the [Rosenbrock function constrained to a disk](https://en.wikipedia.org/wiki/Test_functions_for_optimization#cite_note-12) which attains a global minima at $(x_0^*,x_1^*)=(1.0, 1.0)$. To facilitate testing the functionality offered by BoFire, we label all points inside of the circle $x_0^2+x_1^2\le2$ as 'acceptable' and futher label anything inside of the interesction of this circle and the circle $(x_0-1)^2+(x_1-1)^2\le1.0$ as 'ideal'; points lying outside of these two locations are labeled as "unacceptable."

In [1]:
# Import packages
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain, Outputs, Inputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput, CategoricalOutput, CategoricalInput
from bofire.data_models.objectives.api import MinimizeObjective, MinimizeSigmoidObjective, ConstrainedCategoricalObjective
import numpy as np
import pandas as pd

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Manual setup of the optimization domain

The following cells show how to manually setup the optimization problem in BoFire for didactic purposes.

In [9]:
# Write helper functions which give the objective and the constraints
def rosenbrock(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    return (1 - x["x_0"]) ** 2 + 100 * (x["x_1"] - x["x_0"] ** 2) ** 2

def constraints(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    feasiblity_vector = []
    for _, row in x.iterrows():
        if (row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0) and ((row["x_0"] - 1.0) ** 2 + (row["x_1"] - 1.0) ** 2 <= 1.0):
            feasiblity_vector.append("ideal")
        elif row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0:
            feasiblity_vector.append("acceptable")
        else:
            feasiblity_vector.append("unacceptable")
    return feasiblity_vector

In [158]:
# Set-up the inputs and outputs, use categorical domain just as an example
input_features = Inputs(features=[ContinuousInput(key=f"x_{i}", bounds=(-1.75, 1.75)) for i in range(2)] + [CategoricalInput(key=f"x_3", categories=["0", "1"], allowed=[True, True])])

# here the minimize objective is used, if you want to maximize you have to use the maximize objective.
output_features = Outputs(features=[
        ContinuousOutput(key=f"f_{0}", objective=MinimizeObjective(w=1.)),
        CategoricalOutput(key=f"f_{1}", categories=["unacceptable", "acceptable", "ideal"], objective=ConstrainedCategoricalObjective(categories=["unacceptable", "acceptable", "ideal"], desirability=[False, True, True])), # This function will be associated with learning the categories
        ContinuousOutput(key=f"f_{2}", objective=MinimizeSigmoidObjective(w=1., tp=0.0, steepness=0.5)),
    ]
)

# Create domain
domain1 = Domain(inputs=input_features, outputs=output_features)

# Sample random points
sample_df = domain1.inputs.sample(100)

# Write a function which outputs one continuous variable and another discrete based on some logic
sample_df["f_0"] = rosenbrock(x=sample_df)
sample_df["f_1"] = constraints(x=sample_df)
sample_df["f_2"] = sample_df["x_3"].astype(float) + 1e-2 * np.random.uniform(size=(len(sample_df),))
sample_df.head(5)

,x_0,x_1,x_3,f_0,f_1,f_2
0,-0.149953,-0.704143,1,54.121306,acceptable,1.009579
1,-0.625311,-0.463190,0,75.608036,acceptable,0.001819
2,-0.765853,0.927654,1,14.754710,acceptable,1.006574
3,-1.447047,-0.059688,1,469.801324,unacceptable,1.002428
4,-0.540554,1.090780,1,66.146436,acceptable,1.004633


In [159]:
# Plot the sample df
import math
import plotly.express as px 
fig = px.scatter(sample_df, x="x_0", y="x_1", color="f_1", width=550, height=525, title="Samples with labels")
fig.add_shape(type="circle",
    xref="x", yref="y",
    opacity=0.1,
    fillcolor="red",
    x0=-math.sqrt(2), y0=-math.sqrt(2), x1=math.sqrt(2), y1=math.sqrt(2),
    line_color="red",
)
fig.add_shape(type="circle",
    xref="x", yref="y",
    opacity=0.2,
    fillcolor="LightSeaGreen",
    x0=0, y0=0, x1=2, y1=2,
    line_color="LightSeaGreen",
)
fig.show()

## Evaluate the classification model performance (outside of the optimization procedure)

In [163]:
# Import packages
import bofire.surrogates.api as surrogates
from bofire.data_models.surrogates.api import ClassificationMLPEnsemble
from bofire.surrogates.diagnostics import ClassificationMetricsEnum

# Instantiate the surrogate model 
model = ClassificationMLPEnsemble(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]), lr=0.03, n_epochs=100, hidden_layer_sizes=(4,2,), weight_decay=0.0, batch_size=10, activation="tanh")
surrogate = surrogates.map(model)

# Fit the model to the classification data
cv_df = sample_df.drop(["f_0", "f_2"], axis=1)
cv_df["valid_f_1"] = 1
cv = surrogate.cross_validate(cv_df, folds=3)


c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\bo

In [164]:
# Print results
cv[0].get_metrics(metrics=ClassificationMetricsEnum, combine_folds=True) # print training set performance

,ACCURACY,F1
0,0.68,0.68


In [165]:
cv[1].get_metrics(metrics=ClassificationMetricsEnum, combine_folds=True) # print test set performance

,ACCURACY,F1
0,0.54,0.54


## Setup strategy and ask for candidates



In [166]:
from bofire.data_models.acquisition_functions.api import qEI
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.surrogates.api import BotorchSurrogates, ClassificationMLPEnsemble, MixedSingleTaskGPSurrogate
from bofire.data_models.domain.api import Outputs

strategy_data = SoboStrategy(domain=domain1, 
                             acquisition_function=qEI(), 
                             surrogate_specs=BotorchSurrogates(surrogates=
                                    [
                                        ClassificationMLPEnsemble(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]), lr=0.03, n_epochs=100, hidden_layer_sizes=(4,2,), weight_decay=0.0, batch_size=10, activation="tanh"),
                                        MixedSingleTaskGPSurrogate(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_2")]))
                                    ]
                                )
                            )

strategy = strategies.map(strategy_data)

strategy.tell(sample_df)

In [167]:
candidates = strategy.ask(10)
candidates

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.



,x_0,x_1,x_3,f_1_pred,f_1_sd,f_0_pred,f_2_pred,f_1_unacceptable_prob,f_1_acceptable_prob,f_1_ideal_prob,f_0_sd,f_2_sd,f_1_unacceptable_sd,f_1_acceptable_sd,f_1_ideal_sd,f_0_des,f_2_des,f_1_des
0,0.403369,0.161374,0,acceptable,0.0,0.092455,0.004524,0.399619,0.599358,0.001023,2.192848,0.002932,0.542024,0.541092,0.001150,-0.092455,0.499435,0.600381
1,0.291096,0.091716,1,acceptable,0.0,0.338534,1.005305,0.112541,0.886353,0.001106,2.772511,0.002915,0.197303,0.196632,0.001800,-0.338534,0.376918,0.887459
2,1.319030,1.750000,1,unacceptable,0.0,-2.338611,1.003526,0.688225,0.110699,0.201075,4.714258,0.003410,0.449328,0.240964,0.445632,2.338611,0.377126,0.311775
3,0.086516,-0.000797,0,acceptable,0.0,0.900832,0.004706,0.398625,0.600412,0.000963,2.342997,0.002935,0.542560,0.541685,0.001131,-0.900832,0.499412,0.601375
4,-0.228315,0.046514,0,acceptable,0.0,1.500286,0.004834,0.397428,0.601626,0.000946,2.390450,0.002935,0.541194,0.540336,0.001115,-1.500286,0.499396,0.602572
5,0.076667,-0.004871,1,acceptable,0.0,0.979306,1.005360,0.198561,0.800833,0.000606,2.372284,0.002916,0.237685,0.237178,0.000844,-0.979306,0.376911,0.801439
6,0.308219,0.087674,0,acceptable,0.0,0.222725,0.004599,0.399124,0.599887,0.000989,2.834773,0.002933,0.542427,0.541527,0.001137,-0.222725,0.499425,0.600876
7,0.829716,0.681121,0,unacceptable,0.0,-0.348330,0.003866,0.510435,0.418242,0.071323,1.065240,0.002941,0.496067,0.425085,0.157438,0.348330,0.499517,0.489565
8,-0.219844,0.044418,1,acceptable,0.0,1.494160,1.005372,0.239904,0.759543,0.000553,2.332408,0.002915,0.304143,0.303688,0.000761,-1.494160,0.376910,0.760096
9,-0.063892,0.013738,0,acceptable,0.0,1.097156,0.004757,0.398289,0.600757,0.000954,1.653161,0.002935,0.542274,0.541407,0.001125,-1.097156,0.499405,0.601711


## Check classification of proposed candidates

Use the logic from above to verify the classification values

In [168]:
# Append to the candidates
candidates["f_1_true"] = constraints(x=candidates)

In [169]:
# Print results
candidates[["f_1_pred", "f_1_true"]]

,f_1_pred,f_1_true
0,acceptable,acceptable
1,acceptable,acceptable
2,unacceptable,unacceptable
3,acceptable,acceptable
4,acceptable,acceptable
5,acceptable,acceptable
6,acceptable,acceptable
7,unacceptable,ideal
8,acceptable,acceptable
9,acceptable,acceptable
